# Transfer Learning

### Import Libraries

In [32]:
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub


import tf_keras as tfk                          # needed due to incompatability with tensorflow_hub version
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



### Import Model

In [20]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"      # mobileNetV2
# inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"        # inceptionNetV3

classifier_model = mobilenet_v2     # choose model that will be used for transfer learning

IMAGE_SHAPE = (224, 224)            # shape of the images that will be used

# instantiate classifier model
classifier = tfk.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))  # Specifies the input shape as (224, 224, 3) to match the 3 channels (RGB)
])

In [21]:
classifier.summary()    # summary of the model architecture

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_6 (KerasLayer)  (None, 1001)              3540265   
                                                                 
Total params: 3540265 (13.51 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________


In [22]:
# import imageNet labels (dataset that mobileNet was originally trained on)
labels_path = tfk.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
imagenet_labels[645:655]    # A quick check to inspect some labels from the file and make sure it was imported correctly

array(['matchstick', 'maypole', 'maze', 'measuring cup', 'medicine chest',
       'megalith', 'microphone', 'microwave', 'military uniform',
       'milk can'], dtype='<U30')

## Train with MonkeyPox Dataset

### Hyperparameters

In [23]:
import pathlib

data_root = pathlib.Path("../data/Augmented_Images")    # points to the folder containing the images that will be used for training

batch_size = 32
img_height = 224
img_width = 224

### Training and Validations datasets

In [33]:
# train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),                         # loads images from the data_root directory
  validation_split=0.2,                   # the dataset is split into training and validation sets using an 80-20 split
  subset="training",                      # set this as the training split
  seed=123,                               # sets a seed for reproducibility in splitting the data
  image_size=(img_height, img_width),     # resizes all images to (224, 224) pixels
  batch_size=batch_size
)
# validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),                         # loads images from the data_root directory
  validation_split=0.2,                   # the dataset is split into training and validation sets using an 80-20 split
  subset="validation",                    # set this as the validation split
  seed=123,                               # sets a seed for reproducibility in splitting the data
  image_size=(img_height, img_width),     # resizes all images to (224, 224) pixels
  batch_size=batch_size
)

Found 3192 files belonging to 2 classes.
Using 2554 files for training.
Found 3192 files belonging to 2 classes.
Using 638 files for validation.


In [34]:
class_names = np.array(train_ds.class_names)    # class labels for Monkeypox dataset
print(class_names)

['Monkeypox' 'Others']


### Testing before transfer learning is performed

In [35]:
normalization_layer = tf.keras.layers.Rescaling(1./255)           # normalize pixel values of images from [0, 255] to [0, 1] by dividing
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # normalize training split where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))     # normalize validation split where x—images, y—labels.

In [37]:
AUTOTUNE = tf.data.AUTOTUNE
# prefetch data to improve performance by overlapping data preprocessing and model execution and cache the dataset in memory
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# sanity check by iterating through the train_ds dataset to inspect the shapes of the images and labels in a batch
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [ ]:
# uses the pre-trained MobileNetV2 model to make predictions on the training dataset before transfer learning
result_batch = classifier.predict(train_ds)

# find the index of the highest predicted value for each image and match it with the image
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]  
predicted_class_names

In [ ]:
# plot of predictions made by the model before transfer learning
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

### Define model without final layer

In [38]:
# Load MobileNetV2 from tf.keras.applications 
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # Freeze the base model

# Create the model
num_classes = len(class_names)

## Training 

In [39]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, ConfusionMatrixDisplay

# plot and save confusion matrix
def save_confusion_matrix(true_labels, predicted_labels, class_names, save_path):
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.savefig(save_path)
    plt.close()

# plot and save loss curves
def save_loss_curve(history, save_path):
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss', color='blue')
    plt.plot(history['val_loss'], label='Validation Loss', color='orange')
    plt.title("Training and Validation Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()

# compute and plot evaluation metrics (accuracy, sensitivity, specificity, F1 score)
def save_evaluation_metrics(true_labels, predicted_labels, history, cm, save_path):
    accuracy = history['val_accuracy'][-1]
    sensitivity = recall_score(true_labels, predicted_labels, average='macro')
    specificity = np.mean(np.diag(cm) / (np.diag(cm) + np.sum(cm, axis=0) - np.diag(cm)))
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    metrics = {
        "Accuracy": accuracy,
        "Sensitivity (Recall)": sensitivity,
        "Specificity": specificity,
        "F1-Score": f1
    }

    plt.figure(figsize=(10, 6))
    plt.bar(metrics.keys(), metrics.values(), color=['darkturquoise', 'sandybrown', 'hotpink', 'limegreen'])
    plt.title("Model Evaluation Metrics")
    plt.ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.ylabel("Score")
    plt.savefig(save_path)
    plt.close()
    return metrics

# save classification report
def save_classification_report(true_labels, predicted_labels, class_names, save_path):
    class_report = classification_report(true_labels, predicted_labels, target_names=class_names, digits=4)
    with open(save_path, "w") as f:
        f.write(class_report)

In [31]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import matplotlib.pyplot as plt

# Define the base path for saving models
save_dir = "../saved_models"
os.makedirs(save_dir, exist_ok=True)

# List to store accuracy results for comparison
model_performance = []

# Parameters
NUM_MODELS = 1
# NUM_EPOCHS = 14  # Or any number of epochs you prefer

# TODO: Add function for custom configurations (eg. different optimizer, learning rate, etc.)

# configurations that will be used in training
configs = [
    {"learning_rate": 0.001, "optimizer": "adam", "epochs": 14, "save_metrics": False},
    # {"learning_rate": 0.0001, "optimizer": "adam", "epochs": 50, "save_metrics": False},
    # {"learning_rate": 0.001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
    # {"learning_rate": 0.0001, "optimizer": "sgd", "epochs": 50, "save_metrics": False},
]

for i, config in enumerate(configs):
    print(f"Training model {i + 1}/{len(configs)} with config: {config}")

    # Recreate the model architecture for each loop iteration
    # feature_extractor_layer = hub.KerasLayer(
    #     feature_extractor_model,
    #     input_shape=(224, 224, 3),
    #     trainable=False)

    # model = tfk.Sequential([
    #     feature_extractor_layer,
    #     tfk.layers.Dense(num_classes)
    # ])

    # Recreate the model architecture for each loop iteration
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes)
    ])

    # model.compile(
    #     optimizer=tfk.optimizers.Adam(),
    #     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #     metrics=['accuracy'])
    
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    
    # Select optimizer based on configuration
    if config["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    elif config["optimizer"] == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=config["learning_rate"])
    else:
        raise ValueError(f"Unsupported optimizer: {config['optimizer']}")
    
    # Train the model
    history = model.fit(train_ds, validation_data=val_ds, epochs=config["epochs"])
    
    if(config["save_metrics"] == True):

        # Create subdirectory for this model
        model_subdir = os.path.join(save_dir, f"model_{i + 1}")
        os.makedirs(model_subdir, exist_ok=True)

        # Save the model
        model_path = os.path.join(model_subdir, f"model_{i + 1}.h5")
        model.save(model_path)
        
        # Save training history for analysis later
        history_path = os.path.join(model_subdir, f"history_{i + 1}.npy")
        np.save(history_path, history.history)

        # Save metrics and plots as PNGs

        # Predictions and true labels
        val_predictions = model.predict(val_ds)
        val_predicted_ids = np.argmax(val_predictions, axis=-1)
        true_labels = np.concatenate([y for x, y in val_ds], axis=0)

        # Save confusion matrix
        confusion_matrix_path = os.path.join(model_subdir, "confusion_matrix.png")
        save_confusion_matrix(true_labels, val_predicted_ids, class_names, confusion_matrix_path)

        # Plot and save loss curve
        loss_curve_path = os.path.join(model_subdir, "loss_curve.png")
        save_loss_curve(history.history, loss_curve_path)

        # Calculate and plot metrics
        cm = confusion_matrix(true_labels, val_predicted_ids)
        bar_chart_path = os.path.join(model_subdir, "evaluation_metrics.png")
        save_evaluation_metrics(true_labels, val_predicted_ids, history.history, cm, bar_chart_path)

        # Save classification report
        classification_report_path = os.path.join(model_subdir, "classification_report.txt")
        save_classification_report(true_labels, val_predicted_ids, class_names, classification_report_path)

    # Record the final validation accuracy for comparison
    final_val_acc = history.history['val_accuracy'][-1]
    model_performance.append({
        "model_path": model_path,
        "config": config,
        "validation_accuracy": final_val_acc
    })

    print(f"Model {i + 1} finished training")


# After the loop, print out the results for comparison
model_performance.sort(key=lambda x: x['validation_accuracy'], reverse=True)
print("\nModels ranked by validation accuracy:")
for i, perf in enumerate(model_performance):
    print(f"Model {i + 1}: Config: {perf['config']}, Validation Accuracy: {perf['validation_accuracy']:.4f}")


Training model 1/1 with config: {'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 14, 'save_metrics': False}
Epoch 1/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 26s 261ms/step - accuracy: 0.6733 - loss: 0.6183 - val_accuracy: 0.8323 - val_loss: 0.3728
Epoch 2/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 18s 230ms/step - accuracy: 0.8579 - loss: 0.3449 - val_accuracy: 0.8511 - val_loss: 0.3221
Epoch 3/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 18s 230ms/step - accuracy: 0.8958 - loss: 0.2850 - val_accuracy: 0.8699 - val_loss: 0.2945
Epoch 4/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 19s 232ms/step - accuracy: 0.9114 - loss: 0.2493 - val_accuracy: 0.8762 - val_loss: 0.2765
Epoch 5/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 18s 230ms/step - accuracy: 0.9238 - loss: 0.2245 - val_accuracy: 0.8871 - val_loss: 0.2637
Epoch 6/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 18s 231ms/step - accuracy: 0.9332 - loss: 0.2059 - val_accuracy: 0.8934 - val_loss: 0.2542
Epoch 7/14
80/80 ━━━━━━━━━━━━━━━━━━━━ 18s 230ms/step - accuracy: 0.9393 - loss: 0.1910 - val_accuracy: 0.8934 -

## Testing

In [28]:
model.compile(
  optimizer=tfk.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


In [ ]:
NUM_EPOCHS = 5

# train model
trained_model = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS)

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

### Predictions

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")